## Dataset 

In [1]:
import pandas as pd

In [2]:
df_1 = pd.read_csv("/tf/Classification/Data/BRCA/Using_cbioportal/data_RNA_Seq_v2_expression_median.tsv", sep = '\t')

In [3]:
df_1.head()

,Hugo_Symbol,Entrez_Gene_Id,TCGA-3C-AAAU-01,TCGA-3C-AALI-01,TCGA-3C-AALJ-01,TCGA-3C-AALK-01,TCGA-4H-AAAK-01,TCGA-5L-AAT0-01,TCGA-5L-AAT1-01,TCGA-5T-A9QA-01,...,TCGA-UL-AAZ6-01,TCGA-UU-A93S-01,TCGA-V7-A7HQ-01,TCGA-W8-A86G-01,TCGA-WT-AB41-01,TCGA-WT-AB44-01,TCGA-XX-A899-01,TCGA-XX-A89A-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01
0,LOC100130426,100130426,0.0000,0.0000,0.9066,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,UBE2Q2P3,100133144,16.3644,9.2659,11.6228,12.0894,6.8468,3.9889,0.0000,1.4644,...,0.3992,4.3126,0.0000,5.5624,0.0000,0.0000,14.3858,22.3240,2.2638,6.8865
2,UBE2Q2P3,100134869,12.9316,17.3790,9.2294,11.0799,14.4298,13.6090,10.5949,8.9958,...,14.3720,10.8828,3.0792,14.3711,6.3091,3.2580,21.4409,27.2744,7.2933,24.7795
3,LOC149767,10357,52.1503,69.7553,154.2974,143.8643,84.2128,114.2572,115.9984,107.5628,...,135.6241,136.1288,29.9974,128.3151,53.6278,42.2643,137.7756,64.1427,85.0461,167.5511
4,TIMM23,10431,408.0760,563.8934,1360.8341,865.5358,766.3830,807.7431,1108.3945,1420.5021,...,1570.1445,2886.3965,1721.8816,697.6744,1245.2681,1877.4180,652.7559,722.7208,1140.2801,1003.5668


In [4]:
df_2 = pd.read_csv("/tf/Classification/Data/BRCA/Using_cbioportal/brca_tcga_pan_can_atlas_clinical_data.tsv", sep = '\t')

In [5]:
df_2.head()

,Study ID,Patient ID,Sample ID,Diagnosis Age,Neoplasm Disease Stage American Joint Committee on Cancer Code,American Joint Committee on Cancer Publication Version Type,Aneuploidy Score,Buffa Hypoxia Score,Cancer Type,TCGA PanCanAtlas Cancer Type Acronym,...,9p Status,9q Status,Subtype,Tissue Prospective Collection Indicator,Tissue Retrospective Collection Indicator,Tissue Source Site,Tumor Disease Anatomic Site,Tumor Type,Patient Weight,Winter Hypoxia Score
0,brca_tcga_pan_can_atlas_2018,TCGA-3C-AAAU,TCGA-3C-AAAU-01,55,STAGE X,6TH,19.0,-21.0,Invasive Breast Carcinoma,BRCA,...,Lost,Lost,BRCA_LumA,No,Yes,3C,Breast,Infiltrating Lobular Carcinoma,NaN,-28.0
1,brca_tcga_pan_can_atlas_2018,TCGA-3C-AALI,TCGA-3C-AALI-01,50,STAGE IIB,6TH,22.0,5.0,Invasive Breast Carcinoma,BRCA,...,Lost,NaN,BRCA_Her2,No,Yes,3C,Breast,Infiltrating Ductal Carcinoma,NaN,20.0
2,brca_tcga_pan_can_atlas_2018,TCGA-3C-AALJ,TCGA-3C-AALJ-01,62,STAGE IIB,7TH,13.0,-5.0,Invasive Breast Carcinoma,BRCA,...,Not Called,Gained,BRCA_LumB,No,Yes,3C,Breast,Infiltrating Ductal Carcinoma,NaN,-10.0
3,brca_tcga_pan_can_atlas_2018,TCGA-3C-AALK,TCGA-3C-AALK-01,52,STAGE IA,7TH,4.0,-27.0,Invasive Breast Carcinoma,BRCA,...,Not Called,Not Called,BRCA_LumA,No,Yes,3C,Breast,Infiltrating Ductal Carcinoma,NaN,4.0
4,brca_tcga_pan_can_atlas_2018,TCGA-4H-AAAK,TCGA-4H-AAAK-01,50,STAGE IIIA,7TH,7.0,-27.0,Invasive Breast Carcinoma,BRCA,...,Gained,Gained,BRCA_LumA,Yes,No,4H,Breast,Infiltrating Lobular Carcinoma,NaN,-20.0


In [6]:
Y = df_2[['Patient ID', 'Subtype']]
Y.head()

,Patient ID,Subtype
0,TCGA-3C-AAAU,BRCA_LumA
1,TCGA-3C-AALI,BRCA_Her2
2,TCGA-3C-AALJ,BRCA_LumB
3,TCGA-3C-AALK,BRCA_LumA
4,TCGA-4H-AAAK,BRCA_LumA


In [7]:
Y_patient_code = Y['Patient ID'].tolist()

In [8]:
df_patient_code = df_1.columns[2:]
df_patient_code = [x[:-3] for x in df_patient_code]

In [9]:
ans = [x for x in df_patient_code if x in Y_patient_code]   ## patient ID's that do have PAM50 avaialable

In [10]:
len(ans)

1086

In [11]:
len(Y_patient_code)    ## Number of patient ID's with PAM50 values in df_2

1084

In [12]:
len(df_patient_code)   ## Total Number of patient ID's in df_1

1100

In [13]:
len(set(df_patient_code))   ## Unique patient ID's in df_1

1093

In [14]:
len(set(Y_patient_code))   ## Unique patient Id's with PAM50 in df_2

1084

In [15]:
len(set(ans))     ## Unique patients in df_1 dataset with their PAM_50 in df_2 dataset

1079

## Algorithm (DNP)

In [18]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_1, Y, test_size=0.33, random_state=42)  ## returns numpy array

In [16]:
## Prepare 2 sets S and C
## S initially contains only the bias term and incrementally adds the features
## C set contains all other features which are not contained in S set
## Design a neural network with ReLu as non linear activation

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
import numpy as np

In [76]:
## Input : x - 2D numpy array of (number of samples (N) X Number of features (F))
SUBTYPES = 5

def build_a_subnetwork():
    
    #input_size = x.shape[1]  ## number of features in input
    
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(3, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.01)),  # kernel regulariser are applied to weights
        tf.keras.layers.Dense(2, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.01)),  ## as per research paper we have to take lp regularisation where p = 2
        tf.keras.layers.Dense(2, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.01)),
        tf.keras.layers.Dense(2, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.01)),
        tf.keras.layers.Dense(3)
    ])
    
#     print(model.layers[0].weights)
    
    return model

# build_a_subnetwork()

In [77]:
def getWeightsToTrain(model, k):
    
    for i in range(k):
        model.lay
# tf.Variable(np.random.random(size=(1,)))
a = np.array([[1, 2, 3], [3, 4, 5], [5, 6, 7]])
a[0].shape

(3,)

In [78]:
prev_wts = None

def train_a_network(model, x, y, k):
    
#     if prev_wts != None:
       
    #K.set_value(model.optimizer.learning_rate, 0.001)  ##  initial value of learning rate
    #model.compile(optimizer="Adagrad", loss="CategoricalCrossentropy", metrics=["mae", "acc"])
    #model.fit(x, y)
    
#     wts_to_train = getWeightsToTrain(model, k)
    
    opt = tf.keras.optimizers.Adagrad(learning_rate=0.001)
    loss_fn = lambda: tf.keras.losses.CategoricalCrossentropy(model(input), output)
    var_list = lambda: model.trainable_weights
    
    for input, output in zip(x,y):
        print(input, output)
        opt.minimize(loss_fn, var_list)
    
model = build_a_subnetwork()
train_a_network(model, np.array([[1, 2, 3], [3, 4, 5], [5, 6, 7]]), np.array([1, 2, 1]), 1)

[1 2 3] 1


ValueError: Input 0 of layer sequential_36 is incompatible with the layer: : expected min_ndim=2, found ndim=1. Full shape received: (3,)

In [ ]:
def dnp():
    
    S_size = 1
    C_Size = (X_train.shape[0] + X_test.shape[0])
    x = np.ones((X_train.shape[0], 1))
    
    while(S_size < k):
        model = build_a_subnetwork(x, y_train)
        train_a_network(model, x, y)
        
        
    